In [1]:
# 텐서플로 2 버전 선택
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf
import numpy as np

In [2]:
import tensorflow_hub as hub

inception_url = 'https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4'
feature_model = tf.keras.Sequential([hub.KerasLayer(inception_url,output_shape=(2048,),trainable=False)])
feature_model.build([None,299,299,3])
feature_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     multiple                  21802784  
Total params: 21,802,784
Trainable params: 0
Non-trainable params: 21,802,784
_________________________________________________________________


In [13]:
import os
import shutil
import pandas as pd
import tqdm as tqdm

label_text = pd.read_csv('labels.csv')
os.mkdir('./train_sub')

for i in range(len(label_text)):
    if os.path.exists('./train_sub/' + label_text.loc[i]['breed']) == False:
        os.mkdir('./train_sub/' + label_text.loc[i]['breed'])
    shutil.copy('./train/' + label_text.loc[i]['id'] + '.jpg', './train_sub/' + label_text.loc[i]['breed'])

In [14]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_resnet_v2 import preprocess_input

image_size = 299
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255., horizontal_flip=True, shear_range=0.2, zoom_range=0.2, width_shift_range=0.2, height_shift_range=0.2, validation_split=0.25)
valid_datagen = ImageDataGenerator(rescale=1./255., validation_split=0.25)

train_generator = train_datagen.flow_from_directory(directory="./train_sub/", subset="training", batch_size=batch_size, seed=42, shuffle=True, class_mode="categorical", target_size=(image_size, image_size))
valid_generator = valid_datagen.flow_from_directory(directory="./train_sub/", subset="validation", batch_size=1, seed=42, shuffle=True, class_mode="categorical", target_size=(image_size, image_size))

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
